# 0. Импорты

In [448]:
import math
import pandas as pd
import numpy as np
import re
import nltk
import requests
from urllib import request
from urllib.error import HTTPError, URLError
import unidecode
from bs4 import BeautifulSoup
from itertools import combinations_with_replacement, combinations, product
import tabula
import warnings
from tqdm.notebook import trange, tqdm
from joblib import dump, load
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from fake_useragent import UserAgent
from catboost import CatBoostClassifier
from sklearn.utils import class_weight
pd.set_option('display.max_colwidth', None)



# 3. Исследование

В этой части проекта вы должны подготовить все, что используется в классах и методах, описанных выше, в Jupyter Notebook (recipes.ipynb).

# 3.1. Прогноз

## 1. Подготовка данных

   - Используйте датасет из Epicurious, подготовленный HugoDarwood
   - Отфильтруйте поля: удалите как можно больше столбцов, которые не относятся к названиями ингридиентов. Вы будете прогнозировать оценку или категорию оценки, используя только ингридиенты и ничего больше. Если модель обучится на других данных, которые не будут передаваться на вход, то точность прогнозов будет заведомо ниже.

In [363]:
df = pd.read_csv('epi_r.csv', index_col = False)

In [5]:
pd.set_option('display.max_columns', None)
df.head(5)

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,30 days of groceries,advance prep required,alabama,alaska,alcoholic,almond,amaretto,anchovy,anise,anniversary,anthony bourdain,aperitif,appetizer,apple,apple juice,apricot,arizona,artichoke,arugula,asian pear,asparagus,aspen,atlanta,australia,avocado,back to school,backyard bbq,bacon,bake,banana,barley,basil,bass,bastille day,bean,beef,beef rib,beef shank,beef tenderloin,beer,beet,bell pepper,berry,beverly hills,birthday,biscuit,bitters,blackberry,blender,blue cheese,blueberry,boil,bok choy,bon appétit,bon app��tit,boston,bourbon,braise,bran,brandy,bread,breadcrumbs,breakfast,brie,brine,brisket,broccoli,broccoli rabe,broil,brooklyn,brown rice,brownie,brunch,brussel sprout,buffalo,buffet,bulgaria,bulgur,burrito,butter,buttermilk,butternut squash,butterscotch/caramel,cabbage,cake,california,calvados,cambridge,campari,camping,canada,candy,candy thermometer,cantaloupe,capers,caraway,cardamom,carrot,cashew,casserole/gratin,cauliflower,caviar,celery,chambord,champagne,chard,chartreuse,cheddar,cheese,cherry,chestnut,chicago,chicken,chickpea,chile,chile pepper,chili,chill,chive,chocolate,christmas,christmas eve,cilantro,cinco de mayo,cinnamon,citrus,clam,clove,cobbler/crumble,cocktail,cocktail party,coconut,cod,coffee,coffee grinder,cognac/armagnac,collard greens,colorado,columbus,condiment,condiment/spread,connecticut,cook like a diner,cookbook critic,cookie,cookies,coriander,corn,cornmeal,costa mesa,cottage cheese,couscous,crab,cranberry,cranberry sauce,cream cheese,créme de cacao,crêpe,cr��me de cacao,cuba,cucumber,cumin,cupcake,currant,curry,custard,dairy,dairy free,dallas,date,deep-fry,denver,dessert,digestif,dill,dinner,dip,diwali,dominican republic,dorie greenspan,double boiler,dried fruit,drink,drinks,duck,easter,eau de vie,edible gift,egg,egg nog,eggplant,egypt,emeril lagasse,endive,engagement party,england,entertaining,epi + ushg,epi loves the microwave,escarole,fall,family reunion,fat free,father's day,fennel,feta,fig,fish,flaming hot summer,flat bread,florida,fontina,food processor,fortified wine,fourth of july,france,frangelico,frankenrecipe,freeze/chill,freezer food,friendsgiving,frittata,fritter,frozen dessert,fruit,fruit juice,fry,game,garlic,georgia,germany,gin,ginger,goat cheese,goose,gouda,gourmet,graduation,grains,grand marnier,granola,grape,grapefruit,grappa,green bean,green onion/scallion,grill,grill/barbecue,ground beef,ground lamb,guam,guava,haiti,halibut,halloween,ham,hamburger,hanukkah,harpercollins,hawaii,hazelnut,healdsburg,healthy,herb,high fiber,hollywood,hominy/cornmeal/masa,honey,honeydew,hors d'oeuvre,horseradish,hot drink,hot pepper,house & garden,house cocktail,houston,hummus,ice cream,ice cream machine,iced coffee,iced tea,idaho,illinois,indiana,iowa,ireland,israel,italy,jalapeño,jam or jelly,jamaica,japan,jerusalem artichoke,juicer,jícama,kahlúa,kale,kansas,kansas city,kentucky,kentucky derby,kid-friendly,kidney friendly,kirsch,kitchen olympics,kiwi,kosher,kosher for passover,kumquat,kwanzaa,labor day,lamb,lamb chop,lamb shank,lancaster,las vegas,lasagna,leafy green,leek,legume,lemon,lemon juice,lemongrass,lentil,lettuce,lima bean,lime,lime juice,lingonberry,liqueur,lobster,london,long beach,los angeles,louisiana,louisville,low cal,low carb,low cholesterol,low fat,low sodium,low sugar,low/no sugar,lunar new year,lunch,lychee,macadamia nut,macaroni and cheese,maine,mandoline,mango,maple syrup,mardi gras,margarita,marinade,marinate,marsala,marscarpone,marshmallow,martini,maryland,massachusetts,mayonnaise,meat,meatball,meatloaf,melon,mexico,mezcal,miami,michigan,microwave,midori,milk/cream,minneapolis,minnesota,mint,mississippi,missouri,mixer,molasses,monterey jack,mortar and pestle,mother's day,mozzarella,muffin,mushroom,mussel,mustard,mustard greens,nancy silverton,nebraska,nectarine,new hampshire,new jersey,new mexico,new orleans,new year's day,new year's eve,new york,"no meat, no problem",no sugar a

In [364]:
ingredients = frozenset(['title',
 'rating',
 'alcoholic',
 'almond',
 'amaretto',
 'anchovy',
 'anise',
 'apple',
 'apple juice',
 'apricot',
 'artichoke',
 'asian pear',
 'asparagus',
 'avocado',
 'bacon',
 'banana',
 'basil',
 'bean',
 'beef',
 'beef rib',
 'beef shank',
 'beef tenderloin',
 'beer',
 'beet',
 'bell pepper',
 'berry',
 'biscuit',
 'blackberry',
 'blue cheese',
 'blueberry',
 'bourbon',
 'bran',
 'brandy',
 'bread',
 'breadcrumbs',
 'brie',
 'brine',
 'brisket',
 'broccoli',
 'broccoli rabe',
 'brown rice',
 'brownie',
 'brussel sprout',
 'buffalo',
 'bulgur',
 'burrito',
 'butter',
 'buttermilk',
 'butternut squash',
 'butterscotch/caramel',
 'cabbage',
 'cake',
 'calvados',
 'campari',
 'candy',
 'cantaloupe',
 'capers',
 'caraway',
 'cardamom',
 'carrot',
 'cashew',
 'casserole/gratin',
 'cauliflower',
 'caviar',
 'celery',
 'chambord',
 'champagne',
 'chard',
 'chartreuse',
 'cheddar',
 'cheese',
 'cherry',
 'chestnut',
 'chicken',
 'chickpea',
 'chile pepper',
 'chili',
 'chive',
 'chocolate',
 'cilantro',
 'cinnamon',
 'citrus',
 'clam',
 'clove',
 'cobbler/crumble',
 'cocktail',
 'coconut',
 'cod',
 'coffee',
 'cognac/armagnac',
 'collard greens',
 'condiment',
 'condiment/spread',
 'cookie',
 'cookies',
 'coriander',
 'corn',
 'cornmeal',
 'cottage cheese',
 'couscous',
 'crab',
 'cranberry',
 'cranberry sauce',
 'cream cheese',
 'cucumber',
 'cumin',
 'cupcake',
 'currant',
 'curry',
 'custard',
 'dairy',
 'dill',
 'dried fruit',
 'drink',
 'drinks',
 'duck',
 'egg',
 'egg nog',
 'eggplant',
 'endive',
 'escarole',
 'fall',
 'fennel',
 'feta',
 'fig',
 'fish',
 'flat bread',
 'fontina',
 'fortified wine',
 'frangelico',
 'frittata',
 'fritter',
 'frozen dessert',
 'fruit',
 'fruit juice',
 'garlic',
 'gin',
 'ginger',
 'goat cheese',
 'goose',
 'gouda',
 'grains',
 'granola',
 'grape',
 'grapefruit',
 'grappa',
 'green bean',
 'green onion/scallion',
 'ground beef',
 'ground lamb',
 'guam',
 'guava',
 'halibut',
 'ham',
 'hamburger',
 'hanukkah',
 'harpercollins',
 'hazelnut',
 'herb',
 'honey',
 'honeydew',
 "hors d'oeuvre",
 'horseradish',
 'hot pepper',
 'hummus',
 'ice cream',
 'iced coffee',
 'iced tea',
 'jalapeño',
 'jam or jelly',
 'jerusalem artichoke',
 'jícama',
 'kahlúa',
 'kale',
 'kirsch',
 'kiwi',
 'kumquat',
 'kwanzaa',
 'lamb',
 'lamb chop',
 'lamb shank',
 'lasagna',
 'leafy green',
 'leek',
 'legume',
 'lemon',
 'lemon juice',
 'lemongrass',
 'lentil',
 'lettuce',
 'lima bean',
 'lime',
 'lime juice',
 'lingonberry',
 'liqueur',
 'lobster',
 'lychee',
 'macadamia nut',
 'macaroni and cheese',
 'mandoline',
 'mango',
 'maple syrup',
 'margarita',
 'marinade',
 'marsala',
 'marscarpone',
 'marshmallow',
 'martini',
 'mayonnaise',
 'meat',
 'meatball',
 'meatloaf',
 'melon',
 'mezcal',
 'midori',
 'mint',
 'molasses',
 'mozzarella',
 'muffin',
 'mushroom',
 'mussel',
 'mustard',
 'mustard greens',
 'nectarine',
 'noodle',
 'nut',
 'nutmeg',
 'oat',
 'oatmeal',
 'octopus',
 'okra',
 'olive',
 'omelet',
 'onion',
 'orange',
 'orange juice',
 'oregano',
 'oregon',
 'orzo',
 'oyster',
 'pacific palisades',
 'paleo',
 'pancake',
 'papaya',
 'paprika',
 'parmesan',
 'parsley',
 'parsnip',
 'passion fruit',
 'pasta',
 'pastry',
 'pea',
 'peach',
 'peanut',
 'peanut butter',
 'pear',
 'pecan',
 'pepper',
 'pernod',
 'persimmon',
 'pickles',
 'pie',
 'pine nut',
 'pineapple',
 'pistachio',
 'pizza',
 'plantain',
 'plum',
 'pomegranate',
 'pomegranate juice',
 'poppy',
 'pork',
 'pork chop',
 'pork rib',
 'pork tenderloin',
 'pot pie',
 'potato',
 'potato salad',
 'poultry',
 'poultry sausage',
 'prosciutto',
 'prune',
 'pumpkin',
 'purim',
 'quail',
 'quiche',
 'quince',
 'quinoa',
 'rabbit',
 'rack of lamb',
 'radicchio',
 'radish',
 'raisin',
 'ramadan',
 'ramekin',
 'raspberry',
 'raw',
 'red wine',
 'rhubarb',
 'rice',
 'ricotta',
 'root vegetable',
 'rosemary',
 'rosé',
 'rum',
 'rutabaga',
 'rye',
 'saffron',
 'sake',
 'salad',
 'salad dressing',
 'salmon',
 'sandwich',
 'sangria',
 'sardine',
 'sauce',
 'sausage',
 'sauté',
 'scallop',
 'scotch',
 'seafood',
 'seed',
 'self',
 'semolina',
 'sesame',
 'sesame oil',
 'shallot',
 'shavuot',
 'shellfish',
 'sherry',
 'shrimp',
 'side',
 'simmer',
 'smoothie',
 'snapper',
 'sorbet',
 'sour cream',
 'sourdough',
 'soy',
 'soy sauce',
 'sparkling wine',
 'spice',
 'spinach',
 'squid',
 'steak',
 'strawberry',
 'sugar snap pea',
 'sukkot',
 'sweet potato/yam',
 'swiss cheese',
 'swordfish',
 'taco',
 'tamarind',
 'tangerine',
 'tapioca',
 'tarragon',
 'tart',
 'tea',
 'tequila',
 'thyme',
 'tilapia',
 'tofu',
 'tomatillo',
 'tomato',
 'tortillas',
 'tropical fruit',
 'trout',
 'tuna',
 'turnip',
 'vanilla',
 'veal',
 'vegan',
 'vegetable',
 'venison',
 'vermouth',
 'vinegar',
 'vodka',
 'waffle',
 'walnut',
 'wasabi',
 'watermelon',
 'whiskey',
 'white wine',
 'whole wheat',
 'wild rice',
 'wine',
 'wok',
 'yellow squash',
 'yogurt',
 'zucchini',
 'turkey'])

In [365]:
df_clean = df.drop(columns = [item for item in list(df.columns) if item not in ingredients])

In [366]:
df_clean.head(5)

,title,rating,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,asian pear,asparagus,avocado,bacon,banana,basil,bean,beef,beef rib,beef shank,beef tenderloin,beer,beet,bell pepper,berry,biscuit,blackberry,blue cheese,blueberry,bourbon,bran,brandy,bread,breadcrumbs,brie,brine,brisket,broccoli,broccoli rabe,brown rice,brownie,brussel sprout,buffalo,bulgur,burrito,butter,buttermilk,butternut squash,butterscotch/caramel,cabbage,cake,calvados,campari,candy,cantaloupe,capers,caraway,cardamom,carrot,cashew,casserole/gratin,cauliflower,caviar,celery,chambord,champagne,chard,chartreuse,cheddar,cheese,cherry,chestnut,chicken,chickpea,chile pepper,chili,chive,chocolate,cilantro,cinnamon,citrus,clam,clove,cobbler/crumble,cocktail,coconut,cod,coffee,cognac/armagnac,collard greens,condiment,condiment/spread,cookie,cookies,coriander,corn,cornmeal,cottage cheese,couscous,crab,cranberry,cranberry sauce,cream cheese,cucumber,cumin,cupcake,currant,curry,custard,dairy,dill,dried fruit,drink,drinks,duck,egg,egg nog,eggplant,endive,escarole,fall,fennel,feta,fig,fish,flat bread,fontina,fortified wine,frangelico,frittata,fritter,frozen dessert,fruit,fruit juice,garlic,gin,ginger,goat cheese,goose,gouda,grains,granola,grape,grapefruit,grappa,green bean,green onion/scallion,ground beef,ground lamb,guam,guava,halibut,ham,hamburger,hanukkah,harpercollins,hazelnut,herb,honey,honeydew,hors d'oeuvre,horseradish,hot pepper,hummus,ice cream,iced coffee,iced tea,jalapeño,jam or jelly,jerusalem artichoke,jícama,kahlúa,kale,kirsch,kiwi,kumquat,kwanzaa,lamb,lamb chop,lamb shank,lasagna,leafy green,leek,legume,lemon,lemon juice,lemongrass,lentil,lettuce,lima bean,lime,lime juice,lingonberry,liqueur,lobster,lychee,macadamia nut,macaroni and cheese,mandoline,mango,maple syrup,margarita,marinade,marsala,marscarpone,marshmallow,martini,mayonnaise,meat,meatball,meatloaf,melon,mezcal,midori,mint,molasses,mozzarella,muffin,mushroom,mussel,mustard,mustard greens,nectarine,noodle,nut,nutmeg,oat,oatmeal,octopus,okra,olive,omelet,onion,orange,orange juice,oregano,oregon,orzo,oyster,pacific palisades,paleo,pancake,papaya,paprika,parmesan,parsley,parsnip,passion fruit,pasta,pastry,pea,peach,peanut,peanut butter,pear,pecan,pepper,pernod,persimmon,pickles,pie,pine nut,pineapple,pistachio,pizza,plantain,plum,pomegranate,pomegranate juice,poppy,pork,pork chop,pork rib,pork tenderloin,pot pie,potato,potato salad,poultry,poultry sausage,prosciutto,prune,pumpkin,purim,quail,quiche,quince,quinoa,rabbit,rack of lamb,radicchio,radish,raisin,ramadan,ramekin,raspberry,raw,red wine,rhubarb,rice,ricotta,root vegetable,rosemary,rosé,rum,rutabaga,rye,saffron,sake,salad,salad dressing,salmon,sandwich,sangria,sardine,sauce,sausage,sauté,scallop,scotch,seafood,seed,self,semolina,sesame,sesame oil,shallot,shavuot,shellfish,sherry,shrimp,side,simmer,smoothie,snapper,sorbet,sour cream,sourdough,soy,soy sauce,sparkling wine,spice,spinach,squid,steak,strawberry,sugar snap pea,sukkot,sweet potato/yam,swiss cheese,swordfish,taco,tamarind,tangerine,tapioca,tarragon,tart,tea,tequila,thyme,tilapia,tofu,tomatillo,tomato,tortillas,tropical fruit,trout,tuna,turnip,vanilla,veal,vegan,vegetable,venison,vermouth,vinegar,vodka,waffle,walnut,wasabi,watermelon,whiskey,white wine,whole wheat,wild rice,wine,wok,yellow squash,yogurt,zucchini,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [9]:
df_clean.shape

(20052, 391)

In [10]:
df_clean.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Data columns (total 391 columns):
 #    Column                Non-Null Count  Dtype  
---   ------                --------------  -----  
 0    title                 20052 non-null  object 
 1    rating                20052 non-null  float64
 2    alcoholic             20052 non-null  float64
 3    almond                20052 non-null  float64
 4    amaretto              20052 non-null  float64
 5    anchovy               20052 non-null  float64
 6    anise                 20052 non-null  float64
 7    apple                 20052 non-null  float64
 8    apple juice           20052 non-null  float64
 9    apricot               20052 non-null  float64
 10   artichoke             20052 non-null  float64
 11   asian pear            20052 non-null  float64
 12   asparagus             20052 non-null  float64
 13   avocado               20052 non-null  float64
 14   bacon                 20052 non-null  float64
 15   

In [121]:
df_clean.to_csv('epi_r_clean.csv', index = False)

In [11]:
df_clean.describe()

,rating,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,asian pear,asparagus,avocado,bacon,banana,basil,bean,beef,beef rib,beef shank,beef tenderloin,beer,beet,bell pepper,berry,biscuit,blackberry,blue cheese,blueberry,bourbon,bran,brandy,bread,breadcrumbs,brie,brine,brisket,broccoli,broccoli rabe,brown rice,brownie,brussel sprout,buffalo,bulgur,burrito,butter,buttermilk,butternut squash,butterscotch/caramel,cabbage,cake,calvados,campari,candy,cantaloupe,capers,caraway,cardamom,carrot,cashew,casserole/gratin,cauliflower,caviar,celery,chambord,champagne,chard,chartreuse,cheddar,cheese,cherry,chestnut,chicken,chickpea,chile pepper,chili,chive,chocolate,cilantro,cinnamon,citrus,clam,clove,cobbler/crumble,cocktail,coconut,cod,coffee,cognac/armagnac,collard greens,condiment,condiment/spread,cookie,cookies,coriander,corn,cornmeal,cottage cheese,couscous,crab,cranberry,cranberry sauce,cream cheese,cucumber,cumin,cupcake,currant,curry,custard,dairy,dill,dried fruit,drink,drinks,duck,egg,egg nog,eggplant,endive,escarole,fall,fennel,feta,fig,fish,flat bread,fontina,fortified wine,frangelico,frittata,fritter,frozen dessert,fruit,fruit juice,garlic,gin,ginger,goat cheese,goose,gouda,grains,granola,grape,grapefruit,grappa,green bean,green onion/scallion,ground beef,ground lamb,guam,guava,halibut,ham,hamburger,hanukkah,harpercollins,hazelnut,herb,honey,honeydew,hors d'oeuvre,horseradish,hot pepper,hummus,ice cream,iced coffee,iced tea,jalapeño,jam or jelly,jerusalem artichoke,jícama,kahlúa,kale,kirsch,kiwi,kumquat,kwanzaa,lamb,lamb chop,lamb shank,lasagna,leafy green,leek,legume,lemon,lemon juice,lemongrass,lentil,lettuce,lima bean,lime,lime juice,lingonberry,liqueur,lobster,lychee,macadamia nut,macaroni and cheese,mandoline,mango,maple syrup,margarita,marinade,marsala,marscarpone,marshmallow,martini,mayonnaise,meat,meatball,meatloaf,melon,mezcal,midori,mint,molasses,mozzarella,muffin,mushroom,mussel,mustard,mustard greens,nectarine,noodle,nut,nutmeg,oat,oatmeal,octopus,okra,olive,omelet,onion,orange,orange juice,oregano,oregon,orzo,oyster,pacific palisades,paleo,pancake,papaya,paprika,parmesan,parsley,parsnip,passion fruit,pasta,pastry,pea,peach,peanut,peanut butter,pear,pecan,pepper,pernod,persimmon,pickles,pie,pine nut,pineapple,pistachio,pizza,plantain,plum,pomegranate,pomegranate juice,poppy,pork,pork chop,pork rib,pork tenderloin,pot pie,potato,potato salad,poultry,poultry sausage,prosciutto,prune,pumpkin,purim,quail,quiche,quince,quinoa,rabbit,rack of lamb,radicchio,radish,raisin,ramadan,ramekin,raspberry,raw,red wine,rhubarb,rice,ricotta,root vegetable,rosemary,rosé,rum,rutabaga,rye,saffron,sake,salad,salad dressing,salmon,sandwich,sangria,sardine,sauce,sausage,sauté,scallop,scotch,seafood,seed,self,semolina,sesame,sesame oil,shallot,shavuot,shellfish,sherry,shrimp,side,simmer,smoothie,snapper,sorbet,sour cream,sourdough,soy,soy sauce,sparkling wine,spice,spinach,squid,steak,strawberry,sugar snap pea,sukkot,sweet potato/yam,swiss cheese,swordfish,taco,tamarind,tangerine,tapioca,tarragon,tart,tea,tequila,thyme,tilapia,tofu,tomatillo,tomato,tortillas,tropical fruit,trout,tuna,turnip,vanilla,veal,vegan,vegetable,venison,vermouth,vinegar,vodka,waffle,walnut,wasabi,watermelon,whiskey,white wine,whole wheat,wild rice,wine,wok,yellow squash,yogurt,zucchini,turkey
count,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052

In [367]:
X = df_clean.drop(columns=['rating', 'title'])

In [368]:
y = df_clean['rating']
y.head(2)

0    2.500
1    4.375
Name: rating, dtype: float64

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=21,
                                                    stratify=None)

## 2. Регрессия

   - Для прогнозирования рейтинга опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и оцените RMSE на тестовой подвыборке.

In [16]:
def crossval(n_splits, X, y, model):
    X = np.array(X)
    y = np.array(y)
    kf = KFold(n_splits=n_splits,
                shuffle=True,
                random_state=21)
    result=0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        fit = model.fit(X_train, y_train)
        train_predict = fit.predict(X_train)
        test_predict = fit.predict(X_test)
        rmse_train = mean_squared_error(y_train, train_predict, squared=False)
        rmse_test = mean_squared_error(y_test, test_predict, squared=False)
        print('train -  {}    |    test -  {}'.format(rmse_train, rmse_test))
        result +=rmse_test
    result /= n_splits
    print('Средний RMSE на кросс-валидации -  {}'.format(result))
    return result

*LinearRegression*

In [17]:
X_train_lr = X_train
scaler = StandardScaler()
X_train_lr[list(X_train_lr.columns)] = scaler.fit_transform(X_train_lr); X_train_lr

,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,asian pear,asparagus,avocado,bacon,banana,basil,bean,beef,beef rib,beef shank,beef tenderloin,beer,beet,bell pepper,berry,biscuit,blackberry,blue cheese,blueberry,bourbon,bran,brandy,bread,breadcrumbs,brie,brine,brisket,broccoli,broccoli rabe,brown rice,brownie,brussel sprout,buffalo,bulgur,burrito,butter,buttermilk,butternut squash,butterscotch/caramel,cabbage,cake,calvados,campari,candy,cantaloupe,capers,caraway,cardamom,carrot,cashew,casserole/gratin,cauliflower,caviar,celery,chambord,champagne,chard,chartreuse,cheddar,cheese,cherry,chestnut,chicken,chickpea,chile pepper,chili,chive,chocolate,cilantro,cinnamon,citrus,clam,clove,cobbler/crumble,cocktail,coconut,cod,coffee,cognac/armagnac,collard greens,condiment,condiment/spread,cookie,cookies,coriander,corn,cornmeal,cottage cheese,couscous,crab,cranberry,cranberry sauce,cream cheese,cucumber,cumin,cupcake,currant,curry,custard,dairy,dill,dried fruit,drink,drinks,duck,egg,egg nog,eggplant,endive,escarole,fall,fennel,feta,fig,fish,flat bread,fontina,fortified wine,frangelico,frittata,fritter,frozen dessert,fruit,fruit juice,garlic,gin,ginger,goat cheese,goose,gouda,grains,granola,grape,grapefruit,grappa,green bean,green onion/scallion,ground beef,ground lamb,guam,guava,halibut,ham,hamburger,hanukkah,harpercollins,hazelnut,herb,honey,honeydew,hors d'oeuvre,horseradish,hot pepper,hummus,ice cream,iced coffee,iced tea,jalapeño,jam or jelly,jerusalem artichoke,jícama,kahlúa,kale,kirsch,kiwi,kumquat,kwanzaa,lamb,lamb chop,lamb shank,lasagna,leafy green,leek,legume,lemon,lemon juice,lemongrass,lentil,lettuce,lima bean,lime,lime juice,lingonberry,liqueur,lobster,lychee,macadamia nut,macaroni and cheese,mandoline,mango,maple syrup,margarita,marinade,marsala,marscarpone,marshmallow,martini,mayonnaise,meat,meatball,meatloaf,melon,mezcal,midori,mint,molasses,mozzarella,muffin,mushroom,mussel,mustard,mustard greens,nectarine,noodle,nut,nutmeg,oat,oatmeal,octopus,okra,olive,omelet,onion,orange,orange juice,oregano,oregon,orzo,oyster,pacific palisades,paleo,pancake,papaya,paprika,parmesan,parsley,parsnip,passion fruit,pasta,pastry,pea,peach,peanut,peanut butter,pear,pecan,pepper,pernod,persimmon,pickles,pie,pine nut,pineapple,pistachio,pizza,plantain,plum,pomegranate,pomegranate juice,poppy,pork,pork chop,pork rib,pork tenderloin,pot pie,potato,potato salad,poultry,poultry sausage,prosciutto,prune,pumpkin,purim,quail,quiche,quince,quinoa,rabbit,rack of lamb,radicchio,radish,raisin,ramadan,ramekin,raspberry,raw,red wine,rhubarb,rice,ricotta,root vegetable,rosemary,rosé,rum,rutabaga,rye,saffron,sake,salad,salad dressing,salmon,sandwich,sangria,sardine,sauce,sausage,sauté,scallop,scotch,seafood,seed,self,semolina,sesame,sesame oil,shallot,shavuot,shellfish,sherry,shrimp,side,simmer,smoothie,snapper,sorbet,sour cream,sourdough,soy,soy sauce,sparkling wine,spice,spinach,squid,steak,strawberry,sugar snap pea,sukkot,sweet potato/yam,swiss cheese,swordfish,taco,tamarind,tangerine,tapioca,tarragon,tart,tea,tequila,thyme,tilapia,tofu,tomatillo,tomato,tortillas,tropical fruit,trout,tuna,turnip,vanilla,veal,vegan,vegetable,venison,vermouth,vinegar,vodka,waffle,walnut,wasabi,watermelon,whiskey,white wine,whole wheat,wild rice,wine,wok,yellow squash,yogurt,zucchini,turkey
15035,-0.21201,-0.173736,-0.041061,-0.049368,-0.078402,-0.186645,-0.017658,-0.119004,-0.088621,-0.032572,-0.103803,-0.116281,-0.172399,-0.093494,-0.168917,-0.173164,-0.204515,-0.05304,-0.027361,-0.061274,-0.076776,-0.099738,-0.201183,-0.153231,-0.020894,-0.085347,-0.112078,10.581342,-0.080782,-0.007896,-0.118193,-0.160736,-0.055354,-0.024976,-0.046088,-0.044003,-0.078402,-0.052445,-0.032572,-0.007896,-0.054784,-0.022338,-0.037893,-0.007896,-0.102879,-0.063291,-0.0965,-0.027361,-0.122196,-0.188604,-0.028479,-0.033517,-0.064276,-0.051844,-0.096171,-0.037059,-0.045403,-0.194031,-0.054209,-0.100374,-0.079996,-0.007896,-0.130597,-0.026196,-0.071681,-0.062793,-0.032

In [28]:
lr_cv = crossval(10, X_train_lr, y_train, LinearRegression(positive = True))

train -  1.3016024732827585    |    test -  1.3140855331806396
train -  1.3046960863487564    |    test -  1.2804822308709811
train -  1.3002075882728041    |    test -  1.3220936290235923
train -  1.3001258087243703    |    test -  1.320344563028709
train -  1.30506126809679    |    test -  1.2761913014725754
train -  1.2989027350005402    |    test -  1.3332854044388216
train -  1.30078281331749    |    test -  2.4505172651862708
train -  1.2975620522806874    |    test -  1.3487196227563754
train -  1.2955684566695513    |    test -  5.4290952640999155
train -  1.2996962828338496    |    test -  1.326970861959415
Средний RMSE на кросс-валидации -  1.8401785676017295


*DecisionTreeRegressor*

In [31]:
tree_cv = crossval(10, X_train, y_train, DecisionTreeRegressor(random_state=21))

train -  0.38010756068634977    |    test -  1.666983256626657
train -  0.38689105224991766    |    test -  1.6108349990638775
train -  0.3813037275099529    |    test -  1.6630299864202036
train -  0.37713517389452345    |    test -  1.688969735923275
train -  0.3779640915113285    |    test -  1.6281985324332944
train -  0.38459327903188334    |    test -  1.6344038396625449
train -  0.38167154574074436    |    test -  1.6248635105743914
train -  0.3857601294751632    |    test -  1.645431294442339
train -  0.37851109897698065    |    test -  1.6135550459032397
train -  0.38049964425512683    |    test -  1.5696878311329212
Средний RMSE на кросс-валидации -  1.6345958032182744


*RandomForestRegressor*

In [32]:
forest_cv = crossval(10, X_train, y_train, RandomForestRegressor(random_state=21))

train -  0.5866446546946579    |    test -  1.3159738201806181
train -  0.5915764555299554    |    test -  1.2263049422113523
train -  0.5895972569517393    |    test -  1.3149675237409901
train -  0.586555924843916    |    test -  1.297686773030382
train -  0.5875422147140151    |    test -  1.2646082477570837
train -  0.5908622931490555    |    test -  1.2783154724242989
train -  0.5879541578318503    |    test -  1.287282122228587
train -  0.5927787090728214    |    test -  1.2398820197135418
train -  0.5846550368243029    |    test -  1.328414239083318
train -  0.5860712031754288    |    test -  1.2907075265806518
Средний RMSE на кросс-валидации -  1.2844142686950824


### *DecisionTreeRegressor*

In [33]:
param_grid_dt = {'max_depth': range(4,10),
                 'max_features': ['sqrt', 'log2']}

In [34]:
grid_dt = GridSearchCV(estimator=DecisionTreeRegressor(random_state=21), scoring = 'neg_root_mean_squared_error', param_grid=param_grid_dt)

In [35]:
grid_dt.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeRegressor(random_state=21),
             param_grid={'max_depth': range(4, 10),
                         'max_features': ['sqrt', 'log2']},
             scoring='neg_root_mean_squared_error')

In [36]:
grid_dt.best_params_

{'max_depth': 7, 'max_features': 'sqrt'}

In [40]:
best_dtr = grid_dt.best_estimator_

In [42]:
dump(best_dtr, 'best_dtr.joblib')

['best_dtr.joblib']

In [43]:
rmse_test_dtr = mean_squared_error(y_test, best_dtr.predict(X_test), squared=False); rmse_test_dtr

1.308289806006345

### *RandomForestRegressor*

In [47]:
param_grid_rf = grid = {'n_estimators': [250, 300, 350],
                        'max_depth': [30, 40, 50],
                        'max_features': ['log2']}

In [65]:
grid_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=21), scoring = 'neg_root_mean_squared_error', param_grid=param_grid_rf, verbose=True, n_jobs=-1)

In [49]:
grid_rf.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(estimator=RandomForestRegressor(random_state=21), n_jobs=-1,
             param_grid={'max_depth': [30, 40, 50], 'max_features': ['log2'],
                         'n_estimators': [250, 300, 350]},
             scoring='neg_root_mean_squared_error', verbose=True)

In [50]:
grid_rf.best_params_

{'max_depth': 50, 'max_features': 'log2', 'n_estimators': 300}

In [52]:
best_rfr = grid_rf.best_estimator_

In [54]:
dump(best_rfr, 'best_rfr.joblib')

['best_rfr.joblib']

In [55]:
rmse_test_rfr = mean_squared_error(y_test, best_rfr.predict(X_test), squared=False); rmse_test_rfr

1.3053459763905042

   - Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и оцените RMSE на тестовой подвыборке.

### *Voting Regressor*

In [62]:
param_grid_vr = {'weights': [[1, 2, 3], [3, 2, 1], [1, 2, 1], [4, 2, 3], [5, 2, 1], [1, 5, 1], [1, 2, 5]]}

In [63]:
lr = LinearRegression(positive = True)
dtr = DecisionTreeRegressor(max_depth = 7, max_features ='sqrt', random_state=21)
rfr = RandomForestRegressor(max_depth = 50, max_features = 'log2', n_estimators = 300, random_state=21)
vr = VotingRegressor(estimators=[('LinearRegression', lr), ('DecisionTreeRegression', dtr), ('RandomForestRegression', rfr)])

In [67]:
grid_vr = GridSearchCV(estimator = vr, scoring = 'neg_root_mean_squared_error', param_grid=param_grid_vr, verbose=True, n_jobs=-1)

In [68]:
grid_vr.fit(X_train, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


GridSearchCV(estimator=VotingRegressor(estimators=[('LinearRegression',
                                                    LinearRegression(positive=True)),
                                                   ('DecisionTreeRegression',
                                                    DecisionTreeRegressor(max_depth=7,
                                                                          max_features='sqrt',
                                                                          random_state=21)),
                                                   ('RandomForestRegression',
                                                    RandomForestRegressor(max_depth=50,
                                                                          max_features='log2',
                                                                          n_estimators=300,
                                                                          random_state=21))]),
             n_jobs=-1,
             param_grid={'weights': [[1, 2, 3], [3, 2, 1], [1, 2, 1], [4, 2, 3],
                                     [5, 2, 1], [1, 5, 1], [1, 2, 5]]},
             scoring='neg_root_mean_squared_error', verbose=True)

In [70]:
grid_vr.best_params_

{'weights': [1, 2, 5]}

In [71]:
best_vr = grid_vr.best_estimator_
dump(best_vr, 'best_vr.joblib')

['best_vr.joblib']

In [72]:
rmse_test_vr = mean_squared_error(y_test, best_vr.predict(X_test), squared=False); rmse_test_vr

1.304791378387898

### *Bagging Regressor*

In [73]:
param_grid_br = {'n_estimators': [1, 10, 20, 30, 40, 50, 60]}

br = BaggingRegressor(base_estimator = lr, random_state=21, n_jobs= -1)

grid_br = GridSearchCV(estimator = br, scoring = 'neg_root_mean_squared_error', param_grid=param_grid_br, verbose=True, n_jobs=-1)

In [74]:
grid_br.fit(X_train, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


GridSearchCV(estimator=BaggingRegressor(base_estimator=LinearRegression(positive=True),
                                        n_jobs=-1, random_state=21),
             n_jobs=-1,
             param_grid={'n_estimators': [1, 10, 20, 30, 40, 50, 60]},
             scoring='neg_root_mean_squared_error', verbose=True)

In [75]:
best_br = grid_br.best_estimator_
dump(best_br, 'best_br.joblib')

['best_br.joblib']

In [76]:
rmse_test_br = mean_squared_error(y_test, best_br.predict(X_test), squared=False); rmse_test_br

1.3012110092062814

### Stacking Regressor

In [79]:
param_grid_sr = {'cv': [5, 10, 15, 20],
                'passthrough': [True, False]}

sr = StackingRegressor(estimators=[('DecisionTreeRegression', dtr), ('RandomForestRegression', rfr)], final_estimator=lr)

grid_sr = GridSearchCV(estimator = sr, scoring = 'neg_root_mean_squared_error', param_grid=param_grid_sr, verbose=True, n_jobs=-1)

In [80]:
grid_sr.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=StackingRegressor(estimators=[('DecisionTreeRegression',
                                                      DecisionTreeRegressor(max_depth=7,
                                                                            max_features='sqrt',
                                                                            random_state=21)),
                                                     ('RandomForestRegression',
                                                      RandomForestRegressor(max_depth=50,
                                                                            max_features='log2',
                                                                            n_estimators=300,
                                                                            random_state=21))],
                                         final_estimator=LinearRegression(positive=True)),
             n_jobs=-1,
             param_grid={'cv': [5, 10, 15, 20], 'passthrough': [True, False]},
             scoring='neg_root_mean_squared_error', verbose=True)

In [81]:
best_sr = grid_sr.best_estimator_
dump(best_sr, 'best_sr.joblib')

['best_sr.joblib']

In [82]:
rmse_test_sr = mean_squared_error(y_test, best_sr.predict(X_test), squared=False); rmse_test_sr

1.305686925085824

   - Посчитайте RMSE для наивного регрессора, в котором вы просто высчитали средний рейтинг.
### Наивный регрессор

In [83]:
y_mean = y_test.mean()
y_pred = [y_mean for i in range(len(y_test))]
print(f'RMSE нааивного регрессора {mean_squared_error(y_test, y_pred, squared=False)}')

RMSE нааивного регрессора 1.3072126305093743


## 3. Классификация
   - Бинаризируйте значения целевой переменной путем округления рейтингов до ближайшего целого числа. Это и будут ваши классы.

In [369]:
target = round(y, 0).astype(int)
target

0        2
1        4
2        4
3        5
4        3
        ..
20047    3
20048    4
20049    4
20050    4
20051    4
Name: rating, Length: 20052, dtype: int32

In [370]:
X_train, X_test, y_train, y_test = train_test_split(X, target,
                                                    test_size=0.2,
                                                    random_state=21,
                                                    stratify=target)

   - Для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и посчитайте accuracy на тестовой подвыборке

### SVM

In [12]:
param_grid_svc = {'kernel' : ['linear', 'rbf', 'sigmoid'],
                  'C': [0.01, 0.1, 1, 1.5, 5, 10],
                  'gamma': ['scale', 'auto'],
                  'class_weight': ['balanced', None]}

In [13]:
grid_svc = GridSearchCV(estimator=SVC(random_state=21, probability=True), param_grid=param_grid_svc, n_jobs=-1)

In [14]:
grid_svc.fit(X_train, y_train)

GridSearchCV(estimator=SVC(probability=True, random_state=21), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 1.5, 5, 10],
                         'class_weight': ['balanced', None],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']})

In [16]:
dump(grid_svc.best_estimator_, 'best_SVC.joblib')

['best_SVC.joblib']

In [17]:
print(f'accuracy модели SVC на тестовой подвыборке {accuracy_score(y_test, grid_svc.best_estimator_.predict(X_test))}')

accuracy модели SVC на тестовой подвыборке 0.6861131887309898


###  DecisionTreeClassifier

In [22]:
param_grid_dt = {'max_depth': range(1, 50, 1),
                 'class_weight': ['balanced', None],
                 'criterion': ['entropy', 'gini']}

In [23]:
grid_dt = GridSearchCV(estimator=DecisionTreeClassifier(random_state=21), param_grid=param_grid_dt, n_jobs=-1, cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 50)})

In [46]:
grid_dt.best_params_

{'class_weight': None, 'criterion': 'gini', 'max_depth': 9}

In [24]:
dump(grid_dt.best_estimator_, 'best_DecisionTreeClassifier.joblib')

['best_DecisionTreeClassifier.joblib']

In [25]:
print(f'accuracy модели DecisionTreeClassifier на тестовой подвыборке {accuracy_score(y_test, grid_dt.best_estimator_.predict(X_test))}')

accuracy модели DecisionTreeClassifier на тестовой подвыборке 0.674644727000748


### RandomForestClassifier

In [398]:
param_grid_rfc = {'max_depth': range(1, 50, 10),
                  'class_weight': ['balanced', None],
                  'criterion': ['entropy', 'gini'],
                  'n_estimators': [50, 100, 150, 200]}

In [28]:
grid_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=21, n_jobs=-1), param_grid=param_grid_rfc, verbose=True, n_jobs=-1)
grid_rfc.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=21),
             n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 50, 10),
                         'n_estimators': [50, 100, 150, 200]},
             verbose=True)

In [ ]:
grid_rfc.best_params_

In [29]:
dump(grid_rfc.best_estimator_, 'best_RandomForestClassifier.joblib')

['best_RandomForestClassifier.joblib']

In [30]:
print(f'accuracy модели RandomForestClassifier на тестовой подвыборке {accuracy_score(y_test, grid_rfc.best_estimator_.predict(X_test))}')

accuracy модели RandomForestClassifier на тестовой подвыборке 0.693592620294191


### Accuracy наивного классификатора

   - Рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.

In [32]:
y_mean = round(y_test.mean(), 0)
y_pred = [y_mean for i in range(len(y_test))]
print(f'accuracy нааивного классификатора {accuracy_score(y_test, y_pred)}')

accuracy нааивного классификатора 0.6576913487908252


###   - Снова проведите бинаризацию целевого столбца, преобразовав теперь целые числа в классы bad (0, 1) (невкусное), so-so (2, 3) (нормальное), great (4, 5) (вкусное).

In [371]:
class_target = target.map({0 : 'bad',
                           1 : 'bad',
                           2 : 'so-so',
                           3 : 'so-so',
                           4 : 'great',
                           5 : 'great'})
class_target
class_target.value_counts()

great    15907
so-so     2145
bad       2000
Name: rating, dtype: int64

In [372]:
X_train_cl, X_test_cl, y_train_cl, y_test_cl = train_test_split(X, class_target,
                                                                test_size=0.2,
                                                                random_state=21,
                                                                stratify=class_target)

In [39]:
y_train_cl.value_counts(normalize=True)

great    0.793280
so-so    0.106976
bad      0.099744
Name: rating, dtype: float64

In [40]:
y_test_cl.value_counts(normalize=True)

great    0.793318
so-so    0.106956
bad      0.099726
Name: rating, dtype: float64

   - Снова для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе кроссвалидации и посчитайте accuracy на тестовой подвыборке.

### SVM

In [42]:
param_grid_svc_cl = {'kernel' : ['rbf'],
                  'C': [1.2, 1.5, 1.8],
                  'gamma': ['scale'],
                  'class_weight': [None]}
grid_svc_cl = GridSearchCV(estimator=SVC(random_state=21, probability=True), param_grid=param_grid_svc_cl, n_jobs=-1, cv=5)
grid_svc_cl.fit(X_train_cl, y_train_cl)
grid_svc_cl.best_params_

{'C': 1.8, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}

In [44]:
dump(grid_svc_cl.best_estimator_, 'best_SVC_cl.joblib')

['best_SVC_cl.joblib']

In [45]:
print(f'accuracy модели SVC на тестовой подвыборке {accuracy_score(y_test_cl, grid_svc_cl.best_estimator_.predict(X_test_cl))}')

accuracy модели SVC на тестовой подвыборке 0.8080279232111692


###  DecisionTreeClassifier

In [47]:
param_grid_dt_cl = {'max_depth': range(1, 50, 1),
                 'class_weight': ['balanced', None],
                 'criterion': ['entropy', 'gini']}
grid_dt_cl = GridSearchCV(estimator=DecisionTreeClassifier(random_state=21), param_grid=param_grid_dt_cl, n_jobs=-1, cv=5)
grid_dt_cl.fit(X_train_cl, y_train_cl)
dump(grid_dt_cl.best_estimator_, 'best_DecisionTreeClassifier_cl.joblib')
print(f'accuracy модели DecisionTreeClassifier на тестовой подвыборке {accuracy_score(y_test_cl, grid_dt_cl.best_estimator_.predict(X_test_cl))}')

accuracy модели DecisionTreeClassifier на тестовой подвыборке 0.7968087758663674


### RandomForestClassifier

In [373]:
param_grid_rfc_cl = {'max_depth': [10, 50, 100, 200],
                  'max_features': [10, 50, 100, 200],
                  'n_estimators': [10, 50, 100, 200]}
grid_rfc_cl = GridSearchCV(estimator=RandomForestClassifier(random_state=21, n_jobs=-1), param_grid=param_grid_rfc_cl, verbose=True, n_jobs=-1)
grid_rfc_cl.fit(X_train_cl, y_train_cl)
dump(grid_rfc_cl.best_estimator_, 'best_RandomForestClassifier_cl.joblib')
print(f'accuracy модели RandomForestClassifier на тестовой подвыборке {accuracy_score(y_test_cl, grid_rfc_cl.best_estimator_.predict(X_test_cl))}')

Fitting 5 folds for each of 64 candidates, totalling 320 fits
accuracy модели RandomForestClassifier на тестовой подвыборке 0.8155073547743705


In [375]:
grid_rfc_cl.best_params_

{'max_depth': 100, 'max_features': 10, 'n_estimators': 200}

In [374]:
print(f"precision модели RandomForestClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_rfc_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

precision модели RandomForestClassifier на тестовой подвыборке 0.7923300844664357


### Accuracy наивного классификатора

   - Снова рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.

In [58]:
y_mean_cl = round(y_test_cl.map({'great': 3, 'so-so': 2, 'bad': 1}).mean(), 0)
y_mean_cl

3.0

In [59]:
y_mean_cl = 'great'
y_pred_cl = [y_mean_cl for i in range(len(y_test_cl))]
print(f'accuracy нааивного классификатора {accuracy_score(y_test_cl, y_pred_cl)}')

accuracy нааивного классификатора 0.7933183744702069


###   - Что хуже: спрогнозировать плохой рейтинг, который на самом деле окажется хорошим, или спрогнозировать хороший рейтинг, который на самом деле окажется плохим? Замените метрику accuracy другой соответствующей метрикой.

**так как метрики RMSE и accuracy не дают нам значительного преимущества перед наивным классификатором и нам намного важнее спрогнозировать плохой рейтинг, который на самом деле окажится хорошим, чем спрогнозировать хороший рейтинг, который на самом деле окажется плохим, меняем нашу метрику оценки модели на precision.**

In [63]:
print(f"precision модели SVC на тестовой подвыборке {precision_score(y_test_cl, grid_svc_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

precision модели SVC на тестовой подвыборке 0.8172762250618455


   - Для прогнозирования классов с новой метрикой опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

### SVM

In [65]:
param_grid_svc_cl = {'kernel' : ['rbf', 'sigmoid'],
                  'C': [1.8, 2, 2.5, 5],
                  'gamma': ['scale'],
                  'class_weight': [None]}
grid_svc_cl = GridSearchCV(estimator=SVC(random_state=21, probability=True), param_grid=param_grid_svc_cl, scoring='precision_weighted', n_jobs=-1, cv=5)
grid_svc_cl.fit(X_train_cl, y_train_cl)
grid_svc_cl.best_params_

{'C': 1.8, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}

In [66]:
print(f"precision модели SVC на тестовой подвыборке {precision_score(y_test_cl, grid_svc_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

precision модели SVC на тестовой подвыборке 0.8172762250618455


In [67]:
dump(grid_svc_cl.best_estimator_, 'best_SVC_cl.joblib')

['best_SVC_cl.joblib']

###  DecisionTreeClassifier

In [68]:
param_grid_dt_cl = {'max_depth': range(1, 50, 1),
                 'class_weight': ['balanced', None],
                 'criterion': ['entropy', 'gini']}
grid_dt_cl = GridSearchCV(estimator=DecisionTreeClassifier(random_state=21), param_grid=param_grid_dt_cl, scoring='precision_weighted', n_jobs=-1, cv=5)
grid_dt_cl.fit(X_train_cl, y_train_cl)
print(f"precision модели DecisionTreeClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_dt_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

precision модели DecisionTreeClassifier на тестовой подвыборке 0.7590227880432449


In [451]:
classes = np.unique(y_train_cl)
cw = class_weight.compute_class_weight(class_weight='balanced', classes=classes, y=y_train_cl)
weights = dict(zip(classes,cw))

In [453]:
param_grid_dt_cl = {'max_depth': range(1, 50, 4),
                 'criterion': ['entropy', 'gini']}
grid_dt_cl = GridSearchCV(estimator=DecisionTreeClassifier(random_state=21, class_weight=weights), param_grid=param_grid_dt_cl, scoring='precision_weighted', n_jobs=-1, cv=5)
grid_dt_cl.fit(X_train_cl, y_train_cl)
print(f"precision модели DecisionTreeClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_dt_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

precision модели DecisionTreeClassifier на тестовой подвыборке 0.7458389564049951


In [69]:
dump(grid_dt_cl.best_estimator_, 'best_DecisionTreeClassifier_cl.joblib')

['best_DecisionTreeClassifier_cl.joblib']

### RandomForestClassifier

In [393]:
from sklearn.metrics import make_scorer

In [418]:
def my_scorer(y_true, y_predicted):
    precision = precision_score(y_true, y_predicted, average='micro', labels=['so-so', 'bad'])
    return precision

In [419]:
my_func = make_scorer(my_scorer, greater_is_better=True)

In [420]:
param_grid_rfc_cl = {'max_depth': range(40, 50, 4),
                     'max_features': range(5, 15, 3),
                     'class_weight': [None],
                     'criterion': ['gini'],
                     'n_estimators': range(100, 200, 20)}
grid_rfc_cl = GridSearchCV(estimator=RandomForestClassifier(random_state=21, n_jobs=-1), param_grid=param_grid_rfc_cl, scoring=my_func, cv=5, verbose=True, n_jobs=-1)
grid_rfc_cl.fit(X_train_cl, y_train_cl)
print(f"precision модели RandomForestClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_rfc_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

Fitting 5 folds for each of 60 candidates, totalling 300 fits
precision модели RandomForestClassifier на тестовой подвыборке 0.8118115086745471


In [421]:
precision_score(y_test_cl, grid_rfc_cl.best_estimator_.predict(X_test_cl), average='micro', labels=['so-so', 'bad'])

0.6785714285714286

In [422]:
grid_rfc_cl.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 44,
 'max_features': 5,
 'n_estimators': 180}

In [423]:
dump(grid_rfc_cl.best_estimator_, 'best_RandomForestClassifier_cl_2.joblib')

['best_RandomForestClassifier_cl_2.joblib']

In [376]:
param_grid_rfc_cl = {'max_depth': [10, 50, 100, 200],
                  'max_features': [10, 50, 100, 200],
                  'n_estimators': [10, 50, 100, 200]}
grid_rfc_cl = GridSearchCV(estimator=RandomForestClassifier(random_state=21, n_jobs=-1), param_grid=param_grid_rfc_cl, scoring='precision_weighted', cv=5, verbose=True, n_jobs=-1)
grid_rfc_cl.fit(X_train_cl, y_train_cl)
print(f"precision модели RandomForestClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_rfc_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

Fitting 5 folds for each of 64 candidates, totalling 320 fits
precision модели RandomForestClassifier на тестовой подвыборке 0.8023612082627706


In [379]:
dump(grid_rfc_cl.best_estimator_, 'best_RandomForestClassifier_cl_1.joblib')

['best_RandomForestClassifier_cl_1.joblib']

In [377]:
print(f"precision модели RandomForestClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_rfc_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

precision модели RandomForestClassifier на тестовой подвыборке 0.8023612082627706


In [390]:
precision_score(y_test_cl, grid_rfc_cl.best_estimator_.predict(X_test_cl), average='weighted', labels=['so-so', 'bad'])

0.7670937468184944

In [378]:
grid_rfc_cl.best_params_

{'max_depth': 50, 'max_features': 10, 'n_estimators': 100}

In [71]:
dump(grid_rfc_cl.best_estimator_, 'best_RandomForestClassifier_cl.joblib')

['best_RandomForestClassifier_cl.joblib']

In [455]:
param_grid_rfc_cl = {'max_depth': range(40, 50, 4),
                     'max_features': range(5, 15, 3),
                     'criterion': ['gini'],
                     'n_estimators': range(100, 200, 20)}
grid_rfc_cl = GridSearchCV(estimator=RandomForestClassifier(random_state=21, class_weight=weights, n_jobs=-1), param_grid=param_grid_rfc_cl, scoring='precision_weighted', cv=5, verbose=True, n_jobs=-1)
grid_rfc_cl.fit(X_train_cl, y_train_cl)
print(f"precision модели RandomForestClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_rfc_cl.best_estimator_.predict(X_test_cl), average='weighted')}")

Fitting 5 folds for each of 60 candidates, totalling 300 fits
precision модели RandomForestClassifier на тестовой подвыборке 0.7777648429638533


In [456]:
grid_rfc_cl.best_params_, grid_rfc_cl.best_score_

({'criterion': 'gini',
  'max_depth': 48,
  'max_features': 14,
  'n_estimators': 180},
 0.758895748453763)

In [457]:
dump(grid_rfc_cl.best_estimator_, 'best_RandomForestClassifier_cl_3.joblib')

['best_RandomForestClassifier_cl_3.joblib']

###   - Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

### Voting classifiers

In [25]:
param_grid_vc = {'voting': ['hard', 'soft'], 'weights': [[1, 2, 3], [3, 2, 1], [1, 2, 1], [4, 2, 3], [5, 2, 1], [1, 5, 1], [1, 2, 5]]}

In [23]:
svc_v = load('best_SVC_cl.joblib')
dtc_v = load('best_DecisionTreeClassifier_cl.joblib')
rfc_v = load('best_RandomForestClassifier_cl.joblib')

In [74]:
vc = VotingClassifier(estimators=[('SVC', svc_v), ('DecisionTreeClassifier', dtc_v), ('RandomForestClassifier', rfc_v)])
grid_vc = GridSearchCV(estimator = vc, scoring = 'precision_weighted', param_grid=param_grid_vc, cv=5, verbose=True, n_jobs=-1)
grid_vc.fit(X_train_cl, y_train_cl)
print(f"precision модели Voting classifier на тестовой подвыборке {precision_score(y_test_cl, grid_vc.best_estimator_.predict(X_test_cl), average='weighted')}")

Fitting 5 folds for each of 14 candidates, totalling 70 fits
precision модели Voting classifier на тестовой подвыборке 0.806533351860988


### XGBClassifier

In [17]:
xgbc = XGBClassifier(random_state=21)
param_grid_xgbc = {
    "learning_rate": [0.1, 0.01],
    "max_depth": [2, 3, 4],
    "n_estimators": [100, 200, 300, 400]
}
grid_xgbc = GridSearchCV(estimator = xgbc, scoring = 'precision_weighted', param_grid=param_grid_xgbc, cv=5, verbose=True, n_jobs=-1)
grid_xgbc.fit(X_train_cl, y_train_cl)
print(f"precision модели XGBClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_xgbc.best_estimator_.predict(X_test_cl), average='weighted')}")

Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\anaconda\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\anaconda\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[13:32:46] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
precision модели XGBClassifier на тестовой подвыборке 0.7021775746271913


C:\Users\зщ\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [424]:
grid_xgbc.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}

In [26]:
dump(grid_xgbc.best_estimator_, 'best_XGBClassifier_cl.joblib')

['best_XGBClassifier_cl.joblib']

In [428]:
xgbc = XGBClassifier(random_state=21)
param_grid_xgbc = {
    "learning_rate": [0.01, 0.03, 0.06, 0.1],
    "max_depth": [3, 10, 20],
    "n_estimators": [5, 10, 50]
}
grid_xgbc = GridSearchCV(estimator = xgbc, scoring = my_func, param_grid=param_grid_xgbc, cv=5, verbose=True, n_jobs=-1)
grid_xgbc.fit(X_train_cl, y_train_cl)
print(f"precision модели XGBClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_xgbc.best_estimator_.predict(X_test_cl), average='micro', labels=['so-so', 'bad'])}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits


c:\anaconda\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\anaconda\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[02:11:50] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
precision модели XGBClassifier на тестовой подвыборке 0.5735294117647058


In [430]:
grid_xgbc.best_score_, grid_xgbc.best_params_

(0.6807773109243697,
 {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50})

In [442]:
xgbc = XGBClassifier(random_state=21)
param_grid_xgbc = {
    "max_depth": range(1, 10, 1),
    "gamma": [0.5, 1, 0.01]}
grid_xgbc = GridSearchCV(estimator = xgbc, scoring = my_func, param_grid=param_grid_xgbc, cv=5, verbose=True, n_jobs=-1)
grid_xgbc.fit(X_train_cl, y_train_cl)
print(f"precision модели XGBClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_xgbc.best_estimator_.predict(X_test_cl), average='micro', labels=['so-so', 'bad'])}")

Fitting 5 folds for each of 27 candidates, totalling 135 fits


c:\anaconda\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\anaconda\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[14:44:49] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
precision модели XGBClassifier на тестовой подвыборке 0.65625


In [443]:
grid_xgbc.best_score_, grid_xgbc.best_params_

(0.6458707284468596, {'gamma': 0.5, 'max_depth': 1})

In [447]:
xgbc = XGBClassifier(random_state=21)
param_grid_xgbc = {'learning_rate': [0.1], 'max_depth': [3], 'n_estimators': [50], 'scale_pos_weight': [5.0]}
grid_xgbc = GridSearchCV(estimator = xgbc, scoring = my_func, param_grid=param_grid_xgbc, cv=5, verbose=True, n_jobs=-1)
grid_xgbc.fit(X_train_cl, y_train_cl)
print(f"precision модели XGBClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_xgbc.best_estimator_.predict(X_test_cl), average='micro', labels=['so-so', 'bad'])}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\anaconda\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\anaconda\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[14:57:47] WARNING: ..\src\learner.cc:576: 
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:57:47] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
precision модели XGBClassifier на тестовой подвыборке 0.5735294117647058


### CatBoostClassifier

In [435]:
cbc = CatBoostClassifier()
cbc.fit(X_train_cl, y_train_cl)
print(f"precision модели CatBoostClassifier на тестовой подвыборке {precision_score(y_test_cl, cbc.predict(X_test_cl), average='micro', labels=['so-so', 'bad'])}")

Learning rate set to 0.091157
0:	learn: 1.0127090	total: 10.1ms	remaining: 10.1s
1:	learn: 0.9458402	total: 19.3ms	remaining: 9.61s
2:	learn: 0.8947021	total: 28.9ms	remaining: 9.6s
3:	learn: 0.8516826	total: 38.9ms	remaining: 9.69s
4:	learn: 0.8164547	total: 47.5ms	remaining: 9.45s
5:	learn: 0.7881440	total: 56.2ms	remaining: 9.3s
6:	learn: 0.7627693	total: 64.3ms	remaining: 9.13s
7:	learn: 0.7418922	total: 74.2ms	remaining: 9.2s
8:	learn: 0.7239462	total: 85.6ms	remaining: 9.43s
9:	learn: 0.7077208	total: 95.2ms	remaining: 9.43s
10:	learn: 0.6944432	total: 104ms	remaining: 9.35s
11:	learn: 0.6837251	total: 112ms	remaining: 9.2s
12:	learn: 0.6742853	total: 120ms	remaining: 9.12s
13:	learn: 0.6649339	total: 127ms	remaining: 8.96s
14:	learn: 0.6586674	total: 136ms	remaining: 8.92s
15:	learn: 0.6531957	total: 143ms	remaining: 8.81s
16:	learn: 0.6475583	total: 153ms	remaining: 8.83s
17:	learn: 0.6425369	total: 161ms	remaining: 8.77s
18:	learn: 0.6387427	total: 169ms	remaining: 8.72s
19:	l

### StackingClassifier

In [24]:
param_grid_sc = {'cv': [5, 10, 15, 20],
                'passthrough': [True, False]}
sc = StackingClassifier(estimators=[('DecisionTreeClassifier', dtc_v), ('RandomForestClassifier', rfc_v)], final_estimator=svc_v, n_jobs=-1)
grid_sc = GridSearchCV(estimator = sc, scoring = 'precision_weighted', param_grid=param_grid_sc, cv=5, verbose=True, n_jobs=-1)
grid_sc.fit(X_train_cl, y_train_cl)
print(f"precision модели StackingClassifier на тестовой подвыборке {precision_score(y_test_cl, grid_sc.best_estimator_.predict(X_test_cl), average='weighted')}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
precision модели StackingClassifier на тестовой подвыборке 0.8184340103763009


In [27]:
dump(grid_sc.best_estimator_, 'best_StackingClassifier_cl.joblib')

['best_StackingClassifier_cl.joblib']

## 4. Принятие решения

- Примите решение о том, что лучше использовать: регрессионную модель или классификацию. Сохраните выбранную модель. Вы будете использовать ее в основной программе.

**Так как проще спрогнозировать отдельные классы, выбираем наилучшую модель классификации - RandomForestClassifier**

In [473]:
best_model = load('best_RandomForestClassifier_cl_3.joblib')
best_model.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(class_weight={'bad': 3.341875,
                                     'great': 0.420196463654224,
                                     'so-so': 3.115967365967366},
                       max_depth=48, max_features=14, n_estimators=180,
                       n_jobs=-1, random_state=21)>

In [31]:
print(f"precision модели StackingClassifier на тестовой подвыборке {precision_score(y_test_cl, best_model.predict(X_test_cl), average='weighted')}")

precision модели StackingClassifier на тестовой подвыборке 0.8184340103763009


# 3.2 Пищевая ценность

1. Соберите в датафрейм всю информацию о пищевой ценности продуктов из подготовленного вами и отфильтрованного набора данных (только столбцы с продуктами). Используйте для этого следующий API.

In [59]:
def get_nutritional(food_product):
    food_product = food_product
    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?query=apple&pageSize=2&api_key=ha4hk9SmWuKtltyvR9Rz4ffYmISB4nxlzza3x4ah&query={food_product}'
    res = requests.get(url).json()
    Nutrients = {}
    try:
        x = res['foods'][0]['foodNutrients']
        for i in x:
            Nutrients[i['nutrientName']] = [i['value'], i['unitName']]
        return Nutrients
    except IndexError:
        Nutrients = 0
        return Nutrients
    except KeyError:
        Nutrients = 0
        return Nutrients

In [38]:
def conversion_to_table(Nutrients, food_product):
    cc = pd.DataFrame(list(pp.items()), columns=['nutrition', 'value'])
    cc['measure'] = 0
    cc['food_product'] = food_product
    for i in range(len(cc)):
        x = cc.iloc[[i],[1]].value
        y = x[i]
        a = y[0]
        b = y[1]
        cc.iloc[[i],[1]] = a
        cc.iloc[[i],[2]] = b
        cc.iloc[[i],[3]] = food_product
    return cc

In [44]:
list_nutr = list(X.columns)

In [60]:
df_new = pd.DataFrame()

In [61]:
for i in list_nutr:
    pp = get_nutritional(i)
    if pp != 0:
        cc = conversion_to_table(pp, i)
        df_new = pd.concat([df_new, cc])



In [147]:
df_new

,nutrition,value,measure,food_product
0,Galactose,0.0,G,alcoholic
1,"Fiber, total dietary",0.0,G,alcoholic
2,"Iron, Fe",0.03,MG,alcoholic
3,"Magnesium, Mg",12.0,MG,alcoholic
4,"Phosphorus, P",32.0,MG,alcoholic
...,...,...,...,...
88,Proline,0.596,G,turkey
89,Phytosterols,2.0,MG,turkey
90,"Fatty acids, total trans",0.123,G,turkey
91,"Choline, total",54.0,MG,turkey


In [63]:
df_new.to_csv('df_new.csv', index=False)

In [83]:
# tabula.convert_into('Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.pdf', output_format='csv', pages = '2,3', output_path='Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv')

In [104]:
drv = pd.read_csv('Daily-Reference-Values-_DRVs_-under-the-New-NFL.csv', sep=';', index_col=False)
drv.drop(columns=['Infants through 12 months', 'Children 1 through 3 years', 'Pregnant women and lactating women'], inplace=True)
drv.rename(columns={'Food Component': 'Nutrient', 'Adults and Children >= 4 yers': 'Adults and Children >= 4 years'}, inplace=True)
drv

,Nutrient,Unit of measure,Adults and Children >= 4 years
0,Fat,Grams (g),78
1,Saturated fat,Grams (g),20
2,Cholesterol,Milligrams (mg),300
3,Total carbohydrates,Grams (g),275
4,Sodium,Milligrams (mg),2300
5,Dietary Fiber,Grams (g),28
6,Protein,Grams (g),50
7,Added sugars,Grams (g),50


In [205]:
rdi = pd.read_csv('Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv', sep=';', index_col=False)
rdi.drop(columns=['Infants through 12 months', 'Children 1 through 3 years', 'Pregnant women and lactating women '], inplace=True)
rdi.drop([16, 22, 23, 24, 27], inplace=True); rdi

,Nutrient,Unit of measure,Adults and Children >= 4 years
0,Vitamin A,Micrograms RAE (mcg),900.0
1,Vitamin C,Milligrams (mg),90.0
2,Calcium,Milligrams (mg),1300.0
3,Iron,Milligrams (mg),18.0
4,Vitamin D,Micrograms (mcg),20.0
5,Vitamin E,Milligrams (mg),15.0
6,Vitamin K,Micrograms (mcg),120.0
7,Thiamin,Milligrams (mg),1.2
8,Riboflavin,Milligrams (mg),1.3
9,Niacin,Milligrams NE (mg),16.0


In [206]:
ls_nut = pd.concat([drv, rdi], ignore_index=True)

In [207]:
ls_nut

,Nutrient,Unit of measure,Adults and Children >= 4 years
0,Fat,Grams (g),78.0
1,Saturated fat,Grams (g),20.0
2,Cholesterol,Milligrams (mg),300.0
3,Total carbohydrates,Grams (g),275.0
4,Sodium,Milligrams (mg),2300.0
5,Dietary Fiber,Grams (g),28.0
6,Protein,Grams (g),50.0
7,Added sugars,Grams (g),50.0
8,Vitamin A,Micrograms RAE (mcg),900.0
9,Vitamin C,Milligrams (mg),90.0


In [112]:
df_new['nutrition'].unique()

array(['Galactose', 'Fiber, total dietary', 'Iron, Fe', 'Magnesium, Mg',
       'Phosphorus, P', 'Sodium, Na', 'Protein', 'Ash', 'Fructose',
       'Lactose', 'Carotene, beta', 'Carotene, alpha',
       'Vitamin D (D2 + D3)', 'Cryptoxanthin, beta', 'Lycopene',
       'Copper, Cu', 'Manganese, Mn', 'Vitamin A, IU', 'Retinol',
       'Vitamin C, total ascorbic acid', 'Thiamin', 'Riboflavin',
       'Folate, total', 'Vitamin B-12', 'Vitamin K (phylloquinone)',
       'Folate, food', 'Vitamin B-12, added', 'Cholesterol',
       'Fatty acids, total saturated', 'SFA 8:0', 'SFA 12:0', 'SFA 14:0',
       'PUFA 22:6 n-3 (DHA)', 'MUFA 16:1', 'PUFA 20:5 n-3 (EPA)',
       'PUFA 22:5 n-3 (DPA)', 'Zinc, Zn', 'Selenium, Se',
       'Vitamin E (alpha-tocopherol)',
       'Vitamin D (D2 + D3), International Units', 'Total lipid (fat)',
       'Sucrose', 'Glucose', 'Maltose', 'Water', 'Caffeine',
       'Theobromine', 'Sugars, Total', 'Calcium, Ca', 'Potassium, K',
       'Lutein + zeaxanthin', 'Niacin

In [117]:
Name_nutrient_api = ['Total lipid (fat)', 'Fatty acids, total saturated', 'Cholesterol', 'Carbohydrate, by difference', 'Sodium, Na', 'Fiber, total dietary', 'Protein',
                     'Sugars, added', 'Vitamin A, RAE', 'Vitamin C, total ascorbic acid', 'Calcium, Ca', 'Iron, Fe', 'Vitamin D (D2 + D3)', 'Vitamin E (alpha-tocopherol)',
                     'Vitamin K (phylloquinone)', 'Thiamin', 'Riboflavin', 'Niacin', 'Vitamin B-6', 'Folate, DFE', 'Vitamin B-12', 'Biotin', 'Pantothenic acid',
                     'Phosphorus, P', 'Magnesium, Mg', 'Zinc, Zn', 'Selenium, Se', 'Copper, Cu', 'Manganese, Mn', 'Potassium, K', 'Choline, total']

In [208]:
ls_nut['Name_nutrient_api'] = Name_nutrient_api

In [209]:
ls_nut

,Nutrient,Unit of measure,Adults and Children >= 4 years,Name_nutrient_api
0,Fat,Grams (g),78.0,Total lipid (fat)
1,Saturated fat,Grams (g),20.0,"Fatty acids, total saturated"
2,Cholesterol,Milligrams (mg),300.0,Cholesterol
3,Total carbohydrates,Grams (g),275.0,"Carbohydrate, by difference"
4,Sodium,Milligrams (mg),2300.0,"Sodium, Na"
5,Dietary Fiber,Grams (g),28.0,"Fiber, total dietary"
6,Protein,Grams (g),50.0,Protein
7,Added sugars,Grams (g),50.0,"Sugars, added"
8,Vitamin A,Micrograms RAE (mcg),900.0,"Vitamin A, RAE"
9,Vitamin C,Milligrams (mg),90.0,"Vitamin C, total ascorbic acid"


In [225]:
df_nutrition = df_new.loc[df_new['nutrition'].isin(Name_nutrient_api)]

In [226]:
df_nutrition.to_csv('df_nutrition')

2. Конвертируйте все значения в % от суточной нормы потребления. Сохраняйте только те ингридиенты, которые находятся в этой и этой таблицах

In [227]:
df_nutrition['measure'].unique()

array(['G', 'MG', 'UG'], dtype=object)

In [228]:
len(df_nutrition)

10085

In [229]:
pr_daily_norm = []

In [230]:
for i in range(len(df_nutrition)):
    j = ls_nut.index [ls_nut['Name_nutrient_api']==df_nutrition['nutrition'].values[i]].tolist()
    if  df_nutrition['measure'].values[i] == 'G' and ls_nut['Unit of measure'].values[j] == 'Grams (g)':
        pr_daily_norm.append(df_nutrition['value'].values[i] / float(ls_nut['Adults and Children >= 4 years'].values[j[0]]) * 100)
    if  df_nutrition['measure'].values[i] == 'MG' and ls_nut['Unit of measure'].values[j] == 'Milligrams (mg)':
        pr_daily_norm.append(df_nutrition['value'].values[i] / float(ls_nut['Adults and Children >= 4 years'].values[j[0]]) * 100)
    if  df_nutrition['measure'].values[i] == 'MG' and ls_nut['Unit of measure'].values[j] == 'Milligrams NE (mg)':
        pr_daily_norm.append(df_nutrition['value'].values[i] / float(ls_nut['Adults and Children >= 4 years'].values[j[0]]) * 100)
    if  df_nutrition['measure'].values[i] == 'UG' and ls_nut['Unit of measure'].values[j] == 'Micrograms (mcg)':
        pr_daily_norm.append(df_nutrition['value'].values[i] / float(ls_nut['Adults and Children >= 4 years'].values[j[0]]) * 100)
    if  df_nutrition['measure'].values[i] == 'UG' and ls_nut['Unit of measure'].values[j] == 'Micrograms RAE (mcg)':
        pr_daily_norm.append(df_nutrition['value'].values[i] / float(ls_nut['Adults and Children >= 4 years'].values[j[0]]) * 100)
    if  df_nutrition['measure'].values[i] == 'UG' and ls_nut['Unit of measure'].values[j] == 'Micrograms DFE (mcg)':
        pr_daily_norm.append(df_nutrition['value'].values[i] / float(ls_nut['Adults and Children >= 4 years'].values[j[0]]) * 100)

In [231]:
len(pr_daily_norm)

10085

In [232]:
df_nutrition['pr_daily_norm'] = pr_daily_norm

C:\Users\зщ\AppData\Local\Temp\ipykernel_7436\3231518179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nutrition['pr_daily_norm'] = pr_daily_norm


In [233]:
df_nutrition

,nutrition,value,measure,food_product,pr_daily_norm
1,"Fiber, total dietary",0.0,G,alcoholic,0.000000
2,"Iron, Fe",0.03,MG,alcoholic,0.166667
3,"Magnesium, Mg",12.0,MG,alcoholic,2.857143
4,"Phosphorus, P",32.0,MG,alcoholic,2.560000
5,"Sodium, Na",4.0,MG,alcoholic,0.173913
...,...,...,...,...,...
65,"Vitamin A, RAE",9.0,UG,turkey,1.000000
66,Vitamin E (alpha-tocopherol),0.45,MG,turkey,3.000000
67,Pantothenic acid,0.466,MG,turkey,9.320000
78,"Manganese, Mn",0.051,MG,turkey,2.217391


3. Сохраните измененный датафрейм в CSV-файл, который вы будете использовать в своей основной программе.

In [315]:
df_nutrition.to_csv('df_nutrition.csv', index=False)

# 3.3. Похожие рецепты

1. Для каждого рецепта из набора данных найдите ссылку на сайте epicurious.com и подробную информацию о нем (название рецепта, рейтинг на платформе и URl). Если сделать это не удалось, найдите для данного рецепта похожую ссылку в Интернете.

In [352]:
def pre_processing(title):
    title = title.lower()
    re_not_word = r'[^\w\s]'
    title = re.sub(re_not_word, '', title)
    url = f"https://www.epicurious.com/search/{title.replace(' ', '%20')}?search"
    return url

In [353]:
df = pd.read_csv('epi_r_clean.csv'); df.head(5)

,title,rating,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,asian pear,asparagus,avocado,bacon,banana,basil,bean,beef,beef rib,beef shank,beef tenderloin,beer,beet,bell pepper,berry,biscuit,blackberry,blue cheese,blueberry,bourbon,bran,brandy,bread,breadcrumbs,brie,brine,brisket,broccoli,broccoli rabe,brown rice,brownie,brussel sprout,buffalo,bulgur,burrito,butter,buttermilk,butternut squash,butterscotch/caramel,cabbage,cake,calvados,campari,candy,cantaloupe,capers,caraway,cardamom,carrot,cashew,casserole/gratin,cauliflower,caviar,celery,chambord,champagne,chard,chartreuse,cheddar,cheese,cherry,chestnut,chicken,chickpea,chile pepper,chili,chive,chocolate,cilantro,cinnamon,citrus,clam,clove,cobbler/crumble,cocktail,coconut,cod,coffee,cognac/armagnac,collard greens,condiment,condiment/spread,cookie,cookies,coriander,corn,cornmeal,cottage cheese,couscous,crab,cranberry,cranberry sauce,cream cheese,cucumber,cumin,cupcake,currant,curry,custard,dairy,dill,dried fruit,drink,drinks,duck,egg,egg nog,eggplant,endive,escarole,fall,fennel,feta,fig,fish,flat bread,fontina,fortified wine,frangelico,frittata,fritter,frozen dessert,fruit,fruit juice,garlic,gin,ginger,goat cheese,goose,gouda,grains,granola,grape,grapefruit,grappa,green bean,green onion/scallion,ground beef,ground lamb,guam,guava,halibut,ham,hamburger,hanukkah,harpercollins,hazelnut,herb,honey,honeydew,hors d'oeuvre,horseradish,hot pepper,hummus,ice cream,iced coffee,iced tea,jalapeño,jam or jelly,jerusalem artichoke,jícama,kahlúa,kale,kirsch,kiwi,kumquat,kwanzaa,lamb,lamb chop,lamb shank,lasagna,leafy green,leek,legume,lemon,lemon juice,lemongrass,lentil,lettuce,lima bean,lime,lime juice,lingonberry,liqueur,lobster,lychee,macadamia nut,macaroni and cheese,mandoline,mango,maple syrup,margarita,marinade,marsala,marscarpone,marshmallow,martini,mayonnaise,meat,meatball,meatloaf,melon,mezcal,midori,mint,molasses,mozzarella,muffin,mushroom,mussel,mustard,mustard greens,nectarine,noodle,nut,nutmeg,oat,oatmeal,octopus,okra,olive,omelet,onion,orange,orange juice,oregano,oregon,orzo,oyster,pacific palisades,paleo,pancake,papaya,paprika,parmesan,parsley,parsnip,passion fruit,pasta,pastry,pea,peach,peanut,peanut butter,pear,pecan,pepper,pernod,persimmon,pickles,pie,pine nut,pineapple,pistachio,pizza,plantain,plum,pomegranate,pomegranate juice,poppy,pork,pork chop,pork rib,pork tenderloin,pot pie,potato,potato salad,poultry,poultry sausage,prosciutto,prune,pumpkin,purim,quail,quiche,quince,quinoa,rabbit,rack of lamb,radicchio,radish,raisin,ramadan,ramekin,raspberry,raw,red wine,rhubarb,rice,ricotta,root vegetable,rosemary,rosé,rum,rutabaga,rye,saffron,sake,salad,salad dressing,salmon,sandwich,sangria,sardine,sauce,sausage,sauté,scallop,scotch,seafood,seed,self,semolina,sesame,sesame oil,shallot,shavuot,shellfish,sherry,shrimp,side,simmer,smoothie,snapper,sorbet,sour cream,sourdough,soy,soy sauce,sparkling wine,spice,spinach,squid,steak,strawberry,sugar snap pea,sukkot,sweet potato/yam,swiss cheese,swordfish,taco,tamarind,tangerine,tapioca,tarragon,tart,tea,tequila,thyme,tilapia,tofu,tomatillo,tomato,tortillas,tropical fruit,trout,tuna,turnip,vanilla,veal,vegan,vegetable,venison,vermouth,vinegar,vodka,waffle,walnut,wasabi,watermelon,whiskey,white wine,whole wheat,wild rice,wine,wok,yellow squash,yogurt,zucchini,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [354]:
df_recipes = df[['title', 'rating', 'alcoholic']]
df_recipes = df_recipes.rename(columns={'alcoholic': 'url'}); df_recipes

,title,rating,url
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0
2,Potato and Fennel Soup Hodge,3.750,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0
4,Spinach Noodle Casserole,3.125,0.0
...,...,...,...
20047,Parmesan Puffs,3.125,0.0
20048,Artichoke and Parmesan Risotto,4.375,0.0
20049,Turkey Cream Puff Pie,4.375,0.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0


In [355]:
for i in range(len(df_recipes)):
    df_recipes['url'][i] = pre_processing(title=df_recipes['title'].values[i])


C:\Users\зщ\AppData\Local\Temp\ipykernel_7436\1912192915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recipes['url'][i] = pre_processing(title=df_recipes['title'].values[i])


In [356]:
df_recipes = pd.concat([df_recipes, df.drop(columns=['title', 'rating'])], axis=1)
df_recipes['val'] = df_recipes.loc[:, 'alcoholic':'turkey'].sum(axis=1)

In [357]:
df_recipes

,title,rating,url,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,asian pear,asparagus,avocado,bacon,banana,basil,bean,beef,beef rib,beef shank,beef tenderloin,beer,beet,bell pepper,berry,biscuit,blackberry,blue cheese,blueberry,bourbon,bran,brandy,bread,breadcrumbs,brie,brine,brisket,broccoli,broccoli rabe,brown rice,brownie,brussel sprout,buffalo,bulgur,burrito,butter,buttermilk,butternut squash,butterscotch/caramel,cabbage,cake,calvados,campari,candy,cantaloupe,capers,caraway,cardamom,carrot,cashew,casserole/gratin,cauliflower,caviar,celery,chambord,champagne,chard,chartreuse,cheddar,cheese,cherry,chestnut,chicken,chickpea,chile pepper,chili,chive,chocolate,cilantro,cinnamon,citrus,clam,clove,cobbler/crumble,cocktail,coconut,cod,coffee,cognac/armagnac,collard greens,condiment,condiment/spread,cookie,cookies,coriander,corn,cornmeal,cottage cheese,couscous,crab,cranberry,cranberry sauce,cream cheese,cucumber,cumin,cupcake,currant,curry,custard,dairy,dill,dried fruit,drink,drinks,duck,egg,egg nog,eggplant,endive,escarole,fall,fennel,feta,fig,fish,flat bread,fontina,fortified wine,frangelico,frittata,fritter,frozen dessert,fruit,fruit juice,garlic,gin,ginger,goat cheese,goose,gouda,grains,granola,grape,grapefruit,grappa,green bean,green onion/scallion,ground beef,ground lamb,guam,guava,halibut,ham,hamburger,hanukkah,harpercollins,hazelnut,herb,honey,honeydew,hors d'oeuvre,horseradish,hot pepper,hummus,ice cream,iced coffee,iced tea,jalapeño,jam or jelly,jerusalem artichoke,jícama,kahlúa,kale,kirsch,kiwi,kumquat,kwanzaa,lamb,lamb chop,lamb shank,lasagna,leafy green,leek,legume,lemon,lemon juice,lemongrass,lentil,lettuce,lima bean,lime,lime juice,lingonberry,liqueur,lobster,lychee,macadamia nut,macaroni and cheese,mandoline,mango,maple syrup,margarita,marinade,marsala,marscarpone,marshmallow,martini,mayonnaise,meat,meatball,meatloaf,melon,mezcal,midori,mint,molasses,mozzarella,muffin,mushroom,mussel,mustard,mustard greens,nectarine,noodle,nut,nutmeg,oat,oatmeal,octopus,okra,olive,omelet,onion,orange,orange juice,oregano,oregon,orzo,oyster,pacific palisades,paleo,pancake,papaya,paprika,parmesan,parsley,parsnip,passion fruit,pasta,pastry,pea,peach,peanut,peanut butter,pear,pecan,pepper,pernod,persimmon,pickles,pie,pine nut,pineapple,pistachio,pizza,plantain,plum,pomegranate,pomegranate juice,poppy,pork,pork chop,pork rib,pork tenderloin,pot pie,potato,potato salad,poultry,poultry sausage,prosciutto,prune,pumpkin,purim,quail,quiche,quince,quinoa,rabbit,rack of lamb,radicchio,radish,raisin,ramadan,ramekin,raspberry,raw,red wine,rhubarb,rice,ricotta,root vegetable,rosemary,rosé,rum,rutabaga,rye,saffron,sake,salad,salad dressing,salmon,sandwich,sangria,sardine,sauce,sausage,sauté,scallop,scotch,seafood,seed,self,semolina,sesame,sesame oil,shallot,shavuot,shellfish,sherry,shrimp,side,simmer,smoothie,snapper,sorbet,sour cream,sourdough,soy,soy sauce,sparkling wine,spice,spinach,squid,steak,strawberry,sugar snap pea,sukkot,sweet potato/yam,swiss cheese,swordfish,taco,tamarind,tangerine,tapioca,tarragon,tart,tea,tequila,thyme,tilapia,tofu,tomatillo,tomato,tortillas,tropical fruit,trout,tuna,turnip,vanilla,veal,vegan,vegetable,venison,vermouth,vinegar,vodka,waffle,walnut,wasabi,watermelon,whiskey,white wine,whole wheat,wild rice,wine,wok,yellow squash,yogurt,zucchini,turkey,val
0,"Lentil, Apple, and Turkey Wrap",2.500,https://www.epicurious.com/search/lentil%20apple%20and%20turkey%20wrap%20?search,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1

In [358]:
df_recipes.to_csv('df_recipes.csv', index=False)